# NLP Test

Task: Write two short NLP pipelines in Python. The first one should identify tokens that refer to gold and silver (with separate identifiers for each) as financial commodities. Print out the number of documents in the sample file containing at least one reference to gold and the same for silver. The second pipeline should compute the sentiment of sentences where gold or silver are mentioned as commodities. Print out one overall (full sample file) sentiment value for gold and for silver separately. 


# Task1

The following code is a class that contains two constractors and one method.

*Doc constractor contains docstring.

*init constractor constracts an attribute from path of sample.txt.

*Analyze method reads and parses the text file into documentations. It considers any part of text file that starts with @ or RT     as a documentation. In continue it counts any documentation that contains gold and silver seperately.

In [41]:
class Task1:
    def __doc__(self)->str:
        return 'Class docstring here'
    def __init__(self,path)->None:
        self.p = path
    def Analyze(self)->list:
        '''Function docstring here'''
        
        #Reading the sample.txt.
        with open(self.p,'r') as text_file:
            
            #Parsing it into its lines.
            file = text_file.readlines()
            result = list()
            temp = ''
            for i in file:
                
                #Finding lines that starts with @ or RT then adding it and its following lines to a list as a documentation.
                if i.startswith('@') or i.startswith('RT'):
                    if len(temp)>0:
                        result.append(temp)
                        temp = ''
                        temp += i
                        continue
                temp+=i
            
            #Adding last documentation to the list.
            result.append(temp)
            
            #Counting documentations that contains gold and silver.
            gold = 0
            silver = 0
            for i in result :
                if 'gold' in i or 'Gold' in i:
                    gold+=1
                if 'silver' in i or 'Silver' in i:
                    silver+=1
        
        #Printing out the result by a message.
        print(f'{gold} documentations about gold and {silver} documentations about silver have been found.')

In [42]:
sample = Task1('sample.txt')
sample.Analyze()

170 documentations about gold and 71 documentations about silver have been found.


# Task2 

The following code contains a script and a class.

*The script reads a csv file and makes a dataframe of that. In continue it removes stop words from column of data frame 
 that contains sentences about finance. It does it via a function. at the end it tokenizes the sentences.
 
*The class contains two constractors and one method.

 -Doc contractor contains docstring.
 
 -init contractor constracts two attributes from a directory that refers to a pre trained model and another directory that         refers to sample.txt.
 
 -Show method reads the sample.txt then parses it into its sentences. In continue it predicts the sentiment of each sentence       that contains gold and silver seperately through three sentiments Positive, Negative and Neutral. at the end it reports an       overall sentiment of text file about gold and silver seperately.
 
 Note: In order to predicting the sentiment, I've downloaded a data set from link :
       https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis
       then I've created a tensorflow embedding model on that. It's accuracy is about 80%.
       saved model is provided in the zip file.

In [43]:
#Importing needed modules.
from pandas import read_csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
#Instancing from Tokenizer object.
tokenizer = Tokenizer(oov_token='<00U00>')

In [44]:
#Reading the csv file and making a data frame of that.
file = read_csv('data.csv')
df = file.copy()

#Creating a function to remove stop words from each text.
def Filter(text)->str:
    '''Function docstring'''
    stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    result = [word for word in text.lower().split() if word not in stopwords]
    return ' '.join(result)

#Removing the stop words from data frame sentences.
features = [Filter(note) for note in df.Sentence]

#Tokenizing the sentences.
tokenizer.fit_on_texts(features)

In [45]:
class Task2:
    def __doc__(self)->str:
        return 'class docstring here'
    def __init__(self,path_model,path)->None:
        self.p_m = path_model
        self.p = path
    def Show(self)->None:
        '''Function docstring here'''
        
        #loading the pre trained model.
        model = load_model(self.p_m)
        
        #Reading the sample.txt and parsing it into its sentences. 
        #each line that ends with "." is considered as a sentence.
        gold_sentiments = list()
        silver_sentiments = list()
        with open(self.p) as text_file:
            for note in text_file.read().split('.'):
                if 'gold' in note or 'Gold' in note :
                    
                    #Tokenizing the sentence and then making a sequence of that.
                    seq = tokenizer.texts_to_sequences(note)
                    
                    #Padding the sequence.
                    matrix = pad_sequences(seq,padding='post',truncating='post',maxlen=40)
                    
                    #Predincting the sentiment of sentence.
                    pred = model.predict(matrix)
                    
                    #Finding the high percent of the 'Positive', 'Negative' and 'Neutral' labels.
                    idx = pred.tolist()[0].index(max(pred.tolist()[0]))
                    
                    #Adding its index to related list.
                    gold_sentiments.append(idx)
                    
                #Operation is as same as gold.
                if 'silver' in note or 'Silver' in note :
                    seq = tokenizer.texts_to_sequences(note)
                    matrix = pad_sequences(seq,padding='post',truncating='post',maxlen=40)
                    pred = model.predict(matrix)
                    idx = pred.tolist()[0].index(max(pred.tolist()[0]))
                    silver_sentiments.append(idx)
                    
        #Since 0 refers to positive, 1 refers to negative and 2 refers to neutral, So most repeated label refers to overall 
        #sentiment of the text file.
        counts_gold = list()
        counts_silver = list()
        for lbl in [0,1,2]:
            counts_gold.append(gold_sentiments.count(lbl))
            counts_silver.append(silver_sentiments.count(lbl))
        idx_g = ['Positive','Negative','Neutral'][counts_gold.index(max(counts_gold))]
        idx_s = ['Positive','Negative','Neutral'][counts_silver.index(max(counts_silver))]
        
        #Printing out the result by a message.
        print(f'Overall Sentiment about Gold is {idx_g} and about Silver is {idx_s}.')

In [46]:
sample = Task2('model','sample.txt')
sample.Show()

6/6 [==============================] - 0s 2ms/step


8/8 [==============================] - 0s 1ms/step


5/5 [==============================] - 0s 4ms/step


1/1 [==============================] - 0s 16ms/step


3/3 [==============================] - 0s 5ms/step


2/2 [==============================] - 0s 3ms/step


6/6 [==============================] - 0s 2ms/step
Overall Sentiment about Gold is Negative and about Silver is Negative.


Overall Sentiment about Gold is Negative and about Silver is Negative.

# End